In [1]:
import requests

all_companies = []

for page_index in range(14):
 
    url = f'https://api.hsx.vn/l/api/v1/1/securities/stock?pageIndex={page_index+1}&pageSize=30&alphabet=&sectorId='

    # Gửi yêu cầu GET tới API
    response = requests.get(url)

    response.status_code

    data = response.json()

    data

    codes = [company['code'] for company in data['data']['list']]

    all_companies.extend(codes)

all_companies

['AAA',
 'AAM',
 'AAT',
 'ABR',
 'ABS',
 'ABT',
 'ACB',
 'ACC',
 'ACG',
 'ACL',
 'ADG',
 'ADP',
 'ADS',
 'AGG',
 'AGR',
 'AMD',
 'ANV',
 'APG',
 'APH',
 'ASG',
 'ASM',
 'ASP',
 'AST',
 'BAF',
 'BBC',
 'BCE',
 'BCG',
 'BCM',
 'BFC',
 'BHN',
 'BIC',
 'BID',
 'BKG',
 'BMC',
 'BMI',
 'BMP',
 'BRC',
 'BSI',
 'BSR',
 'BTP',
 'BTT',
 'BVH',
 'BWE',
 'C32',
 'C47',
 'CCC',
 'CCI',
 'CCL',
 'CDC',
 'CHP',
 'CIG',
 'CII',
 'CKG',
 'CLC',
 'CLL',
 'CLW',
 'CMG',
 'CMV',
 'CMX',
 'CNG',
 'COM',
 'CRC',
 'CRE',
 'CSM',
 'CSV',
 'CTD',
 'CTF',
 'CTG',
 'CTI',
 'CTR',
 'CTS',
 'CVT',
 'D2D',
 'DAH',
 'DAT',
 'DBC',
 'DBD',
 'DBT',
 'DC4',
 'DCL',
 'DCM',
 'DGC',
 'DGW',
 'DHA',
 'DHC',
 'DHG',
 'DHM',
 'DIG',
 'DLG',
 'DMC',
 'DPG',
 'DPM',
 'DPR',
 'DQC',
 'DRC',
 'DRH',
 'DRL',
 'DSC',
 'DSE',
 'DSN',
 'DTA',
 'DTL',
 'DTT',
 'DVP',
 'DXG',
 'DXS',
 'DXV',
 'EIB',
 'ELC',
 'EVE',
 'EVF',
 'EVG',
 'FCM',
 'FCN',
 'FDC',
 'FIR',
 'FIT',
 'FLC',
 'FMC',
 'FPT',
 'FRT',
 'FTS',
 'GAB',
 'GAS',
 'GDT',


In [2]:
import csv
import time
from bs4 import BeautifulSoup

years = list(range(2005, 2025))

# Hàm tải nội dung với retry
def get_with_retry(url, max_retries=2, wait=2):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36"
    }

    for attempt in range(1, max_retries + 1):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            print(f"    ✅ Tải thành công sau {attempt} lần thử")
            return response
        except requests.exceptions.HTTPError as e:
            print(f"    ⚠️ Lỗi HTTP {response.status_code} - thử lại sau {wait} giây (lần {attempt})")
        except requests.exceptions.RequestException as e:
            print(f"    ❌ Lỗi kết nối: {e} - thử lại sau {wait} giây (lần {attempt})")

        time.sleep(wait)

    print("    ❌ Không thể tải trang sau nhiều lần thử.")
    return None

# Bắt đầu xử lý từng công ty
for ticker in all_companies:
    print(f"🔍 Đang xử lý công ty: {ticker}")

    final_table = []

    for year_index, year in enumerate(years):
        url = f'https://cafef.vn/du-lieu/bao-cao-tai-chinh/{ticker}/bsheet/{year}/4/0/0/bao-cao-tai-chinh-.chn'
        print(f"  -> Năm {year}")

        response = get_with_retry(url)
        if response is None:
            continue  # bỏ qua năm này nếu không tải được

        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', id='tableContent')

        if not table:
            print(f"    ❌ Không tìm thấy bảng dữ liệu")
            continue

        rows = table.find_all('tr')
        current_year_data = []

        for i, row in enumerate(rows):
            cols = row.find_all('td')
            if not cols:
                continue

            if len(cols) > 8:
                cols = cols[:-8]
            else:
                continue

            row_data = [col.text.strip() for col in cols]

            if len(row_data) < 2:
                continue

            if i == 0:
                if year_index == 0:
                    for q in range(1, len(row_data)):
                        if row_data[q] == '':
                            row_data[q] = f"Quý {q} năm {year}"
                    final_table.append(row_data)
                else:
                    new_titles = [f"Quý {q} năm {year}" for q in range(1, len(row_data))]
                    final_table[0].extend(new_titles)
            else:
                if year_index == 0:
                    current_year_data.append(row_data)
                else:
                    current_year_data.append(row_data[1:])  # bỏ cột đầu (chỉ tiêu)

        if year_index == 0:
            final_table.extend(current_year_data)
        else:
            for i in range(1, len(final_table)):
                final_table[i].extend(current_year_data[i - 1])

    # Ghi ra file CSV
    output_file = f"report 1/Tổng hợp báo cáo {ticker}.csv"
    with open(output_file, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(final_table)

    print(f"✅ Đã lưu vào file: {output_file}\n")


🔍 Đang xử lý công ty: AAA
  -> Năm 2005
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2006
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2007
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2008
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2009
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2010
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2011
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2012
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2013
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2014
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2015
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2016
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2017
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2018
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2019
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2020
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2021
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2022
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2023
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2024
    ✅ Tải thành công sau 1 lầ

In [3]:
# Hàm tải nội dung với retry
def get_with_retry(url, max_retries=2, wait=2):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36"
    }

    for attempt in range(1, max_retries + 1):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            print(f"    ✅ Tải thành công sau {attempt} lần thử")
            return response
        except requests.exceptions.HTTPError as e:
            print(f"    ⚠️ Lỗi HTTP {response.status_code} - thử lại sau {wait} giây (lần {attempt})")
        except requests.exceptions.RequestException as e:
            print(f"    ❌ Lỗi kết nối: {e} - thử lại sau {wait} giây (lần {attempt})")

        time.sleep(wait)

    print("    ❌ Không thể tải trang sau nhiều lần thử.")
    return None

# Bắt đầu xử lý từng công ty
for ticker in all_companies:
    print(f"🔍 Đang xử lý công ty: {ticker}")

    final_table = []

    # Bắt đầu xử lý các năm
    for year_index, year in enumerate(years):
        url = f'https://cafef.vn/du-lieu/bao-cao-tai-chinh/{ticker}/incsta/{year}/4/0/0/bao-cao-tai-chinh-.chn'
        print(f"  -> Năm {year}")

        response = get_with_retry(url)
        if response is None:
            continue  # bỏ qua năm này nếu không tải được

        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', id='tableContent')

        if not table:
            print(f"    ❌ Không tìm thấy bảng dữ liệu")
            continue

        rows = table.find_all('tr')
        current_year_data = []

        for i, row in enumerate(rows):
            cols = row.find_all('td')
      
            if not cols:
                continue

            # Bỏ cột cuối và cột đầu tiên như trong code cũ
            if len(cols) > 8:
                cols = cols[:-8]
            else:
                continue

            # Xử lý row_data cho năm 2005 và các năm tiếp theo
            if year == 2005:
                row_data = [col.text.strip() for col in cols]
            else:
                row_data = [col.text.strip() for col in cols[1:]]
                
            if len(row_data) < 2:
                continue

            
            current_year_data.append(row_data)

        # Chỉ thêm dữ liệu năm sau vào final_table, giữ nguyên dữ liệu của năm đầu
        if year_index == 0:
            final_table.extend(current_year_data)
        else:
            for i in range(0, len(final_table)):
                final_table[i].extend(current_year_data[i])

    # Tạo new_row chứa thông tin từ Quý 1 năm 2005 đến Quý 4 năm 2024
    new_row = ['']  # Để cột đầu tiên trống
    for year in years:
        for q in range(1, 5):
            new_row.append(f"Quý {q} năm {year}")

    # Thêm new_row vào đầu bảng (chỉ thêm một lần, không bị lệch)
    final_table.insert(0, new_row)

    # Ghi ra file CSV
    output_file = f"report 2/Tổng hợp báo cáo {ticker} 2.csv"
    with open(output_file, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(final_table)

    print(f"✅ Đã lưu vào file: {output_file}\n")


🔍 Đang xử lý công ty: AAA
  -> Năm 2005
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2006
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2007
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2008
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2009
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2010
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2011
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2012
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2013
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2014
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2015
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2016
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2017
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2018
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2019
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2020
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2021
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2022
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2023
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2024
    ✅ Tải thành công sau 1 lầ

In [4]:
import yfinance as yf
import pandas as pd
import json

max_retries = 2
    
for ticker_symbol in all_companies:
    print(f"Đang xử lý:  {ticker_symbol}")
    
    success = False
    retries = 0
    while not success and retries < max_retries:
        try:
            # Lấy dữ liệu từ 2005 đến 2024 từ mã cổ phiếu ban đầu
            ticker = yf.Ticker(ticker_symbol)
            data = ticker.history(start="2005-01-01", end="2024-12-31", interval="1d").iloc[:, :5]
            print("⚠️ Thử mã với đuôi .VN.")
            ticker_symbol_vn = ticker_symbol + ".VN"
            ticker = yf.Ticker(ticker_symbol_vn)
            try:
                data_vn = ticker.history(start="2005-01-01", end="2024-12-31", interval="1d")
            except Exception as e:
                print(f"❌ Lỗi khi lấy dữ liệu cho {ticker_symbol} : {e}")

            # Gộp dữ liệu từ mã gốc và mã .VN nếu có dữ liệu
            if data_vn is not None and not data_vn.empty:
                print(f"✅ Đã tìm thấy dữ liệu cho {ticker_symbol} với mã {ticker_symbol}.VN.")
                data = pd.concat([data, data_vn.iloc[:, :5]])
                data = data.loc[~data.index.duplicated(keep='last')]  # Loại bỏ các dòng trùng lặp
                data_vn = None
    
            # Dữ liệu không trống, tiếp tục dù số dòng ít
            if data.empty:
                print(f"⚠️ Không có dữ liệu cho {ticker_symbol}. Bỏ qua.")
                break

            # Làm sạch dữ liệu, chuyển định dạng cột Date (bỏ giờ và timezone)
            data = data.reset_index()
            data['Date'] = pd.to_datetime(data['Date'], utc=True).dt.date
            data = data.set_index('Date')

            # Lưu vào file CSV
            filename = f"stock data/{ticker_symbol}_stock_data.csv"
            data.to_csv(filename)

            print(f"✅ Đã lưu dữ liệu cho {ticker_symbol} ({len(data)} dòng).")
            data = None
            success = True  # Gắn cờ thành công
    
        except Exception as e:
            retries += 1
            print(f"❌ Lỗi khi lấy dữ liệu cho {ticker_symbol} (thử lại {retries}/{max_retries}): {e}")
            if retries >= max_retries:
                print(f"⚠️ Đã thử {max_retries} lần mà không thành công. Bỏ qua {ticker_symbol}.")
            time.sleep(2)  # Delay ngắn giữa các lần thử lại
    
    time.sleep(3)  # Delay giữa các công ty
    
# Lưu danh sách công ty hợp lệ và không hợp lệ
print("\n✅ Hoàn thành.")

Đang xử lý:  AAA
⚠️ Thử mã với đuôi .VN.
✅ Đã tìm thấy dữ liệu cho AAA với mã AAA.VN.
✅ Đã lưu dữ liệu cho AAA (1440 dòng).
Đang xử lý:  AAM
⚠️ Thử mã với đuôi .VN.
✅ Đã tìm thấy dữ liệu cho AAM với mã AAM.VN.
✅ Đã lưu dữ liệu cho AAM (427 dòng).
Đang xử lý:  AAT
⚠️ Thử mã với đuôi .VN.
✅ Đã tìm thấy dữ liệu cho AAT với mã AAT.VN.
✅ Đã lưu dữ liệu cho AAT (3875 dòng).
Đang xử lý:  ABR
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho ABR : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho ABR (5032 dòng).
Đang xử lý:  ABS
⚠️ Thử mã với đuôi .VN.
✅ Đã tìm thấy dữ liệu cho ABS với mã ABS.VN.
✅ Đã lưu dữ liệu cho ABS (550 dòng).
Đang xử lý:  ABT
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho ABT : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho ABT (5032 dòng).
Đang xử lý:  ACB
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho ACB : Length mismatch: Expected axis has 2 elements, new values have 

$ACC: possibly delisted; no timezone found


⚠️ Thử mã với đuôi .VN.
✅ Đã tìm thấy dữ liệu cho ACC với mã ACC.VN.
✅ Đã lưu dữ liệu cho ACC (356 dòng).


C:\Users\hype1\AppData\Local\Temp\ipykernel_1248\4230196432.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  data = pd.concat([data, data_vn.iloc[:, :5]])


Đang xử lý:  ACG
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho ACG : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho ACG (1 dòng).
Đang xử lý:  ACL
⚠️ Thử mã với đuôi .VN.
✅ Đã tìm thấy dữ liệu cho ACL với mã ACL.VN.
✅ Đã lưu dữ liệu cho ACL (2787 dòng).
Đang xử lý:  ADG
⚠️ Thử mã với đuôi .VN.
✅ Đã tìm thấy dữ liệu cho ADG với mã ADG.VN.
✅ Đã lưu dữ liệu cho ADG (2972 dòng).
Đang xử lý:  ADP
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho ADP : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho ADP (5032 dòng).
Đang xử lý:  ADS


...

Đang xử lý:  VNL
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho VNL : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho VNL (2966 dòng).
Đang xử lý:  VNM
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho VNM : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho VNM (3870 dòng).
Đang xử lý:  VNS
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho VNS : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho VNS (2983 dòng).
Đang xử lý:  VOS
⚠️ Thử mã với đuôi .VN.
✅ Đã tìm thấy dữ liệu cho VOS với mã VOS.VN.
✅ Đã lưu dữ liệu cho VOS (3235 dòng).
Đang xử lý:  VPB
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho VPB : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho VPB (1135 dòng).
Đang xử lý:  VPD
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho VPD : Length mismatch: Expected axis has 2 elements, new values have 

$VSI: possibly delisted; no timezone found


⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho VSI : Length mismatch: Expected axis has 2 elements, new values have 1 elements
⚠️ Không có dữ liệu cho VSI. Bỏ qua.
Đang xử lý:  VTB
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho VTB : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho VTB (2986 dòng).
Đang xử lý:  VTO
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho VTO : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho VTO (2983 dòng).
Đang xử lý:  VTP


$VTP: possibly delisted; no price data found  (1d 2005-01-01 -> 2024-12-31)


⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho VTP : Length mismatch: Expected axis has 2 elements, new values have 1 elements
⚠️ Không có dữ liệu cho VTP. Bỏ qua.
Đang xử lý:  YBM
⚠️ Thử mã với đuôi .VN.
❌ Lỗi khi lấy dữ liệu cho YBM : Length mismatch: Expected axis has 2 elements, new values have 1 elements
✅ Đã lưu dữ liệu cho YBM (877 dòng).
Đang xử lý:  YEG
⚠️ Thử mã với đuôi .VN.
✅ Đã tìm thấy dữ liệu cho YEG với mã YEG.VN.
✅ Đã lưu dữ liệu cho YEG (2544 dòng).

✅ Hoàn thành.
